In [4]:
# 8-Puzzle Solver Code
import heapq

# Class representing a node in the puzzle
class PuzzleNode:
    def __init__(self, state, parent=None, move=None, cost=0, depth=0):
        self.state = state            # Current state of the puzzle
        self.parent = parent          # Parent node (previous state)
        self.move = move              # Move made to reach this state
        self.cost = cost              # Heuristic cost of this state
        self.depth = depth            # Number of moves made so far

    # Comparison functions for priority queue
    def __lt__(self, other):
        return (self.cost + self.depth) < (other.cost + other.depth)

    def __eq__(self, other):
        return self.state == other.state

    def __hash__(self):
        return hash(str(self.state))

# Heuristic function: Manhattan distance
# Calculates how far tiles are from their goal positions
def heuristic(state, goal):
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                x, y = divmod(goal.index(state[i][j]), 3)
                distance += abs(x - i) + abs(y - j)
    return distance

# Generate all possible neighboring states by sliding the empty tile (0)
def get_neighbors(state):
    neighbors = []
    # Find position of the empty tile
    x, y = next((i, j) for i, row in enumerate(state) for j, val in enumerate(row) if val == 0)
    # Possible moves
    moves = [(x-1, y, 'Up'), (x+1, y, 'Down'), (x, y-1, 'Left'), (x, y+1, 'Right')]

    for nx, ny, move in moves:
        if 0 <= nx < 3 and 0 <= ny < 3:
            # Create a new state by swapping empty tile with adjacent tile
            new_state = [row[:] for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append((new_state, move))

    return neighbors

# A* search algorithm to solve the 8-puzzle
def solve_8_puzzle(start, goal):
    goal_flat = [num for row in goal for num in row]  # Flatten goal state for easy indexing
    start_node = PuzzleNode(start, cost=heuristic(start, goal_flat))
    frontier = [start_node]  # Priority queue for A* search
    explored = set()         # Set of visited states

    while frontier:
        current = heapq.heappop(frontier)  # Get state with lowest cost

        if current.state == goal:  # Goal state reached
            moves = []
            while current.parent:
                moves.append(current.move)
                current = current.parent
            return moves[::-1]  # Reverse moves to get correct order

        explored.add(current)

        for neighbor, move in get_neighbors(current.state):
            neighbor_node = PuzzleNode(neighbor, current, move, heuristic(neighbor, goal_flat), current.depth + 1)
            if neighbor_node not in explored and neighbor_node not in frontier:
                heapq.heappush(frontier, neighbor_node)

    return None  # No solution found

# Main function to execute the solver
if __name__ == "__main__":
    start = [[1, 2, 3], [4, 0, 5], [6, 7, 8]]  # Starting configuration
    goal = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]   # Goal configuration
    solution = solve_8_puzzle(start, goal)

    if solution:
        print(f"Solution found in {len(solution)} moves: {' -> '.join(solution)}")
    else:
        print("No solution found")


Solution found in 14 moves: Right -> Down -> Left -> Left -> Up -> Right -> Down -> Right -> Up -> Left -> Left -> Down -> Right -> Right
